# SiDi-CC Neural Network Event identification

## Introduction

The objective of this study is to improve the event classification and the prediction on the position and energy of the Compton event inside the Compton Camera with the use of a neural network.

The goal is to improve the currently used cut-based reconstruction of the compton event by the predictions of the neural network. The ultimate target of the neural network is to correctly identify compton events, predict the energy and position of the compton event and most importantly, reproduce a range shift in the input data.

## Goal

1. Student (Alexander): Improvements of Awals work, check of variable distributions, increase selection efficiency to something better than random selection. why does the network not reproduce a range shift
2. Student (George): test implementation (by Mareike) and improvement of Convolutional Network on 4-to-1 coupling simulated data
3. Student (Phillippe): Simple network to predict Energy in single fibres from SiPM signals

## Current Solutions

**Awal Awal:**
The current solution is a supervised hybrid Neural Network developed during the Master-thesis of Awal Awal. The network consists of two layer Convolutional-Neural-Network (CNN) followd up by a multi-component Deep-Neural-Network. The NN showcased a higher purity and position resolution than the cut-based reconstruction, but ultimately failed to reproduce a simulated Bragg peak shift.

**Pascal Wirtz:**
The network structure was updated by splitting the prediction tasks to (independent?) networks. No improvement was seen besides a reduction in computing time of around 80%. A confinement parameter was added to handle protons and electrons that do not deposite their entire energy in the Module. No improvement on this side was seen. The network was tested on a new dataset, featuring a more realistic proton simulation. The network can not handle the new coincidence types added by the new dataset.

## Neural Network Performance

The performance of the NN is measured in:

1. $ \textbf{Purity} = \frac{\text{No. correctly reconstructed Compton events}}{\text{No. identified Compton events}} $


2. $ \textbf{Efficiency} = \frac{\text{No. correctly reconstructed Compton events}}{\text{No. Compton events in data}} $


3. **Mean Euclidean Distance:** Mean euclidean distance between the true electron and proton position and the model predicted positions

## Documentation

Full documentation of this problem will be available in this markdown documentation. The code will be available on Github.

# Dataset (Root files)

## Origin

Simulated by Jonas, see PhD Thesis Chapter 4.3 for more information. Proton beam is based on clinical conditions, Real coincidences and Random coincidences (+ pile up) are simulated by event mixing. Major task will be to handle the random coincidences as they build the background of the measurement.

## Root Files:

- `OptimisedGeometry_BP0mm_2e10protons.root`
- `OptimisedGeometry_BP5mm_4e9protons.root`

## Sensitive Attributes

A summary of the used attributes from the **Events** tree:

| **Root Leave** | **Type** | **Description** |
|---|:---:|:---|
| MCEnergy_Primary | Float | MC energy of the primary generated particle |
| MCEnergy_e | Float | MC energy of electron, 0 if no Compton event occured |
| MCEnergy_p | Float | MC energy of photon, 0 if no compton event occured|
| MCSimulatedEventType | Int | 2: Real coincidence <br>3: Random coincidence <br>5: Real coincidence + pile up <br>6: Random coincidence + pile up |
| MCPosition_source | TVector3 | MC position of source |
| MCDirection_source | TVector3 | MC direction of source |
| MCComptonPosition | TVector3 | MC position of Compton event, 0 if no Compton event occured |
| MCDirectionScatter | TVector3 | MC direction after scattering |
| MCPosition_e | TVector3 | MC position of electron |
| MCInteraction_e | Array like<br>(Int) | Interaction type |
| MCPosition_p | TVector3 | MC position of photon |
| MCInteraction_p | Array like<br>(Int) | Interaction type |
| Identified | Boolean | 0: not identified <br>Else: identified|
| PurCrossed | Boolean | not used anymore |
| RecoClusterPositions.position | Array like<br>(TVector3) | Array of cluster positions |
| RecoClusterPositions.uncertainty | TVector3 | Array of cluster position uncertainties |
| RecoClusterEnergies.values | float | Array of cluster energies |
| RecoClusterEnergies.uncertainty | float | Array of cluster energy uncertainty |
| RecoClusterEntries | Array like<br>(Int) | Number of fibres hit per cluster |

A summary of the used attributes from the **Events** tree:

| **Root Leave** | **Type** | **Description** |
|---|:---:|:---|
| ScattererThickness_x | ??? |  |
| ScattererThickness_y | ??? |  |
| ScattererThickness_z | ??? |  |
| AbsorberThickness_x | ??? |  |
| AbsorberThickness_y | ??? |  |
| AbsorberThickness_z | ??? |  |
| ScattererPosition | TVector3 | Position of the centre? of the scatterer module |
| AbsorberPosition | TVector3 | Position of the centre? of the absorber module |

## Event classification

### Awal Basis

- **is_distributed_cluster:** <br>At least one cluster in eah module
- **is_compton:** <br>Is electron energy unequal 0
- **is_complete_compton:** <br>At least 2 p interactions: 0 < interaction type < 10 (first not included) <br>At least 1 e interaction: 10 <= interaction type < 20
- **is_complete_distributed_compton:** <br>e and p secondary event in a different module (bug in script?)
- **is_ideal_compton:** <br>complete distributed compton event with additional tag:<br>**is_ep:** e in scatterer, p in absorber, event_type = 2<br>**is_pe:** p in scatterer, e in absorber, event_type = 2


# Code Base

## Class documentation

Some small notes for code structure:
- root_file.py: contains root file paths
- Event.py: class, containing all event data
- Preprocessing: class, take root file, gives Training data